# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import gmaps 
import requests 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "city_data.csv" 
city_data_df = pd.read_csv(csv_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
cordinates = city_data_df[['Latitude','Longitude']]
cordinates

,Latitude,Longitude
0,56.40,61.88
1,55.10,-105.28
2,38.42,-121.42
3,42.00,8.92
4,67.45,153.68
...,...,...
592,-6.43,39.55
593,37.94,22.96
594,54.01,33.96
595,37.57,126.98


In [5]:
figure_layout = {
    'width' : '400px',
    'height' : '300px',
    'border' : '1px solid black',
    'padding' :'1px',
    'margin' : '0 auto 0 auto'
}

fig = gmaps.figure(layout = figure_layout)

In [7]:
markers = gmaps.marker_layer(cordinates)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
city_data_df = city_data_df.loc[city_data_df['Temperature °F']>70]
city_data_df = city_data_df.loc[city_data_df['Temperature °F']<80]
city_data_df = city_data_df.loc[city_data_df['Wind Speed']<10]
city_data_df = city_data_df.loc[city_data_df['Cloudiness']==0]

In [9]:
hotel_df = city_data_df.dropna()
hotel_df

,Unnamed: 0,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date
19,19,Gerāsh,76.73,35,0,3.09,27.67,54.14,IR,1601261905
123,123,Bianzhuang,74.12,51,0,5.48,34.85,118.04,CN,1601261937
263,263,Erzin,77.00,84,0,0.83,36.96,36.20,TR,1601261977
281,281,Damietta,78.80,78,0,9.17,31.42,31.81,EG,1601261982
305,305,Caravelas,71.31,90,0,7.16,-17.71,-39.25,BR,1601261989
372,372,Buraidah,78.80,39,0,9.17,26.33,43.98,SA,1601261826
393,393,Dalbandin,78.28,15,0,3.27,28.89,64.41,PK,1601262025
412,412,Esmeraldas,75.20,44,0,6.20,-19.76,-44.31,BR,1601261836
485,485,Khirkiya,76.82,77,0,2.59,22.17,76.86,IN,1601262062
531,531,Linhares,70.12,85,0,5.10,-19.39,-40.07,BR,1601262082


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_cordinates = zip(hotel_df['Latitude'],hotel_df['Longitude']) 
target_radius = 5000
target_type = 'lodging'

hotel_name = []

for hotel_cordinates in hotel_cordinates:
    
    try:
        params = {
            'location' : (f'{hotel_cordinates[0]}, {hotel_cordinates[1]}'),
            'radius' : target_radius,
            'type' : target_type,
            'key' : g_key
        }

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        hotel_name.append(hotel_data['results'][0]['name'])
    
    except IndexError:
        print(f'{hotel_cordinates[0]}, {hotel_cordinates[1]} Does not exist')
hotel_name

['Chelpasin',
 'Hao Shuo Business Hotel',
 'Artemis Otel',
 'العواد لتجارة الابلكاش',
 'Pousada dos Navegantes',
 'Mövenpick Hotel Qassim',
 'Police station anti NARCOTICS FORCE',
 'Hotel Fazenda Lagoa Azul',
 'Sanjay Industries',
 'Hotel Conceição',
 'Pousada Aconchego',
 'بورتو بردية',
 'Apart Hotel Rio Xingu']

In [11]:
hotel_df['Hotel Name'] = hotel_name
hotel_df

,Unnamed: 0,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date,Hotel Name
19,19,Gerāsh,76.73,35,0,3.09,27.67,54.14,IR,1601261905,Chelpasin
123,123,Bianzhuang,74.12,51,0,5.48,34.85,118.04,CN,1601261937,Hao Shuo Business Hotel
263,263,Erzin,77.00,84,0,0.83,36.96,36.20,TR,1601261977,Artemis Otel
281,281,Damietta,78.80,78,0,9.17,31.42,31.81,EG,1601261982,العواد لتجارة الابلكاش
305,305,Caravelas,71.31,90,0,7.16,-17.71,-39.25,BR,1601261989,Pousada dos Navegantes
372,372,Buraidah,78.80,39,0,9.17,26.33,43.98,SA,1601261826,Mövenpick Hotel Qassim
393,393,Dalbandin,78.28,15,0,3.27,28.89,64.41,PK,1601262025,Police station anti NARCOTICS FORCE
412,412,Esmeraldas,75.20,44,0,6.20,-19.76,-44.31,BR,1601261836,Hotel Fazenda Lagoa Azul
485,485,Khirkiya,76.82,77,0,2.59,22.17,76.86,IN,1601262062,Sanjay Industries
531,531,Linhares,70.12,85,0,5.10,-19.39,-40.07,BR,1601262082,Hotel Conceição


In [13]:
locations = hotel_df[['Latitude','Longitude']]

fig = gmaps.figure()

humidity = hotel_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10, 
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Latitude','Longitude']]

In [46]:
#location_tuples = locations.itertuples()  # locations is a dataframe
#location_tuples
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add marker layer ontop of heat map

fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))